In [1]:
import pandas as pd
import numpy as np
import json
import os
from scipy.stats import norm
import ast
import re

def top_players_by_position(player_data, relevant_attributes, position, n):
    df_player_data = pd.read_excel(player_data).drop(columns=['maximum_sense_score', 'minimum_sense_score', 'Average Sense Score', 'minimum_pp_score','total_absolute_score', 'average_preferred_position_score'])
    df2= df_player_data.loc[df_player_data["total_game_time"]>= 90]
    df= df2.drop(list(df2)[1:7], axis=1)
    df_relevant_attributes = pd.read_csv(relevant_attributes)
    df_relevant_attributes['relevant_attributes'] = df_relevant_attributes['relevant_attributes'].apply(lambda lst: [s + '_per_90_percentile' for s in ast.literal_eval(lst)])
        
    for col in df.columns:
        df[f'{col}_z_score'] = (df[col] - df[col].mean()) / df[col].std(ddof=0)
        
    for col in df.columns:
        if col.endswith('_z_score'):
            percentile_col = col.replace('_z_score', '_percentile')
            df[percentile_col] = norm.cdf(df[col]) * 100
        
    df.drop(columns=[col for col in df.columns if col.endswith('_z_score')], inplace=True)
    
    cols_to_use= (df.columns.difference(df2.columns)).tolist()
    cols_to_use.append("user_id")
    df3= df2.merge(df[cols_to_use], on='user_id', how='left')
    df3.drop(columns= "user_id_percentile", inplace= True)
    
    def calculate_sum(df_relevant_attributes, df3, position):
        position_row = df_relevant_attributes[df_relevant_attributes['preferred_position'] == position]
    
        if position_row.empty:
            print(f"No columns specified for position {position}")
            return pd.DataFrame()  # Return empty DataFrame if no relevant attributes are found
    
        columns = position_row['relevant_attributes'].iloc[0]
    
        filtered_rows = df3[df3['preferred_position'].apply(lambda x: position in x)]
    
        if filtered_rows.empty:
            print(f"No rows with position {position} found in data")
            return pd.DataFrame()  # Return empty DataFrame if no rows are found
    
        filtered_rows['sum'] = filtered_rows[columns].sum(axis=1)
    
        return filtered_rows
    
    result_df = calculate_sum(df_relevant_attributes, df3, position)
    
    if result_df is not None and not result_df.empty:
        df_filtered = result_df.sort_values(by='sum', ascending=False)
        top_players = df_filtered.head(n)

        def get_relevant_columns(df_relevant_attributes, df2, position):
            relevant_attributes = df_relevant_attributes[df_relevant_attributes['preferred_position'] == position]['relevant_attributes'].iloc[0]
            relevant_columns = [attr.rstrip("_percentile") for attr in relevant_attributes]
            column_order = df2.iloc[:, :7].columns.tolist() + relevant_columns
            return column_order
    
        relevant_columns = get_relevant_columns(df_relevant_attributes, df2, position)
        top_players = top_players[relevant_columns]

        return top_players
    
    return pd.DataFrame()  # Return empty DataFrame if no top players are found

position_list= ['GOALKEEPER','CENTER BACK','LEFT SIDE BACK','RIGHT SIDE BACK','DEFENSIVE MIDFIELDER','CENTRAL MIDFIELDER','LEFT WINGER','RIGHT WINGER',
'ATTACKING MIDFIELDER','CENTER FORWARD']

# Call the function and write the results to Excel
with pd.ExcelWriter("top_players_u15.xlsx", engine='openpyxl') as writer:
    sheet_written = False
    for i in position_list:
        # top_players_df = top_players_by_position(r"C:\Users\LENOVO\Documents\Downloads\dpdl_u15.xlsx", r"C:\Users\LENOVO\relevant_attributes_positionwise.csv", i, 10)
        top_players_df = top_players_by_position(f'{folder}/{sub_folder}/{file_name}.xlsx', "relevant_attributes_positionwise.csv", i, 5)
        if not top_players_df.empty:  # Check if the DataFrame is not empty
            sheet_name = re.sub(r'[\\/*?:[\]]', '', i)[:31]  # Ensure valid sheet name
            top_players_df.to_excel(writer, sheet_name=sheet_name, index=False)
            sheet_written = True
    
    if not sheet_written:
        # If no sheets are written, create an empty sheet to avoid Excel errors
        pd.DataFrame({"Message": ["No data available for any position"]}).to_excel(writer, sheet_name="No Data", index=False)

print("Excel file with multiple worksheets has been created successfully.")


IndexError: At least one sheet must be visible

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_10728\10319516.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[percentile_col] = norm.cdf(df[col]) * 100
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_10728\10319516.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[percentile_col] = norm.cdf(df[col]) * 100
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_10728\10319516.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Cons

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_10728\10319516.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[percentile_col] = norm.cdf(df[col]) * 100
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_10728\10319516.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[percentile_col] = norm.cdf(df[col]) * 100
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_10728\10319516.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Cons

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_10728\10319516.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[percentile_col] = norm.cdf(df[col]) * 100
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_10728\10319516.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[percentile_col] = norm.cdf(df[col]) * 100
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_10728\10319516.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Cons

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_10728\10319516.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[percentile_col] = norm.cdf(df[col]) * 100
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_10728\10319516.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[percentile_col] = norm.cdf(df[col]) * 100
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_10728\10319516.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Cons

Excel file with multiple worksheets has been created successfully.
